In [ ]:
"""Courtesy:
Data_Processing:https://www.kaggle.com/maksimeren/covid-19-literature-clustering
Using BERT Embeddings: https://medium.com/swlh/a-simple-guide-on-using-bert-for-text-classification-bbf041ac8d04
"""

In [1]:
#!pip install kaggle
import os

#!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge


In [2]:
#!unzip  CORD-19-research-challenge.zip -d CORD-19-research-challenge/

In [4]:
import glob
import json
import pandas as pd
from tqdm import tqdm
root_path = '/home/csgrad/hiteshna/.kaggle/CORD-19-research-challenge/'
all_json = glob.glob(f'{root_path}/document_parses/pdf_json/*.json')    #retrieving all the pdf_json files only
len(all_json)

134354

In [5]:
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head()

/home/csgrad/hiteshna/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,5,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [6]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

397ab50fc7fc1e65bc71f9856dcec950c5a249c3: Aim: To estimate time to getting off supplemental oxygen therapy and identify predictors among COVID-19 patients admitted to Millennium COVID-19 Care Center in Addis Ababa, Ethiopia.
Methods: A prospe... The 2019 corona virus disease was first identified in China on December 2019 and later spread to the entire world and declared to be a pandemic by world health organization on March 11, 2020 1 . In Et...


In [7]:
def get_breaks(content, length):
    data = ""
    words = content.split(' ')
    total_chars = 0

    # add break every length characters
    for i in range(len(words)):
        total_chars += len(words[i])
        if total_chars > length:
            data = data + "<br>" + words[i]
            total_chars = 0
        else:
            data = data + " " + words[i]
    return data

In [8]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'abstract_summary': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    # no metadata, skip this paper
    if len(meta_data) == 0:
        continue
    
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    
    #create a column for the summary of abstract to be used in a plot
    if len(content.abstract) == 0: 
        # no abstract provided
        dict_['abstract_summary'].append("Not provided.")
    elif len(content.abstract.split(' ')) > 100:
        # abstract provided is too long for plot, take first 300 words append with ...
        info = content.abstract.split(' ')[:100]
        summary = get_breaks(' '.join(info), 40)
        dict_['abstract_summary'].append(summary + "...")
    else:
        # abstract is short enough
        summary = get_breaks(content.abstract, 40)
        dict_['abstract_summary'].append(summary)
        
    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    
    try:
        # if more than one author
        authors = meta_data['authors'].values[0].split(';')
        if len(authors) > 2:
            # more than 2 authors, may be problem when plotting, so take first 2 append with ...
            dict_['authors'].append(". ".join(authors[:2]) + "...")
        else:
            # authors will fit in plot
            dict_['authors'].append(". ".join(authors))
    except Exception as e:
        # if only one author - or Null valie
        dict_['authors'].append(meta_data['authors'].values[0])
    
    # add the title information, add breaks when needed
    try:
        title = get_breaks(meta_data['title'].values[0], 40)
        dict_['title'].append(title)
    # if title was not provided
    except Exception as e:
        dict_['title'].append(meta_data['title'].values[0])
    
    # add the journal information
    dict_['journal'].append(meta_data['journal'].values[0])
    
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'abstract_summary'])
df_covid.head()

Processing index: 0 of 134354
Processing index: 13435 of 134354
Processing index: 26870 of 134354
Processing index: 40305 of 134354
Processing index: 53740 of 134354
Processing index: 67175 of 134354
Processing index: 80610 of 134354
Processing index: 94045 of 134354
Processing index: 107480 of 134354
Processing index: 120915 of 134354
Processing index: 134350 of 134354


,paper_id,abstract,body_text,authors,title,journal,abstract_summary
0,397ab50fc7fc1e65bc71f9856dcec950c5a249c3,Aim: To estimate time to getting off supplemen...,The 2019 corona virus disease was first identi...,"Leulseged, T. W.. Hassen, I. S....",Duration of Oxygen Requirement and Predictors...,NaN,Aim: To estimate time to getting off<br>suppl...
1,5b6020d44b406b856ea26326d2e1763925050bf9,The serologic response of individuals with mil...,A novel human coronavirus that is now named se...,"Fafi-Kremer, S.. Bruel, T....",Serologic responses to SARS-CoV-2 infection<b...,NaN,The serologic response of individuals with<br...
2,003d88ea27bb48a2cfa96e491dbd4a9740b041bb,Effective property prediction methods can help...,The race to identify promising repurposing dru...,"Jin, Wengong. Barzilay, Regina...",Adaptive Invariance for Molecule Property<br>...,NaN,Effective property prediction methods can<br>...
3,372947b39005f7105cf7ecd4ce618174a655f9be,"In this paper, we study the effectiveness of t...","In December 2019, a novel strand of Coronaviru...","Cooper, Ian. Mondal, Argha...",A SIR model assumption for the spread of<br>C...,Chaos Solitons Fractals,"In this paper, we study the effectiveness of ..."
4,11d016e4272c8a89fe7986afde61e6fb91908304,Interferences between pathogenic bacteria and ...,Otitis media is a common childhood disease; it...,"Chonmaitree, Tasnee. Jennings, Kristofer...",Nasopharyngeal microbiota in infants and<br>c...,PLoS One,Interferences between pathogenic bacteria<br>...


In [9]:
import pickle

In [11]:
df_covid.to_pickle('df_covid.pkl')

In [10]:
len(df_covid)

119594

In [13]:
df_covid.shape

(119594, 7)

In [14]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid['abstract'].describe(include='all')

count     119366
unique     81631
top             
freq       37187
Name: abstract, dtype: object

In [16]:

df_covid.dropna(inplace=True)
df_covid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105889 entries, 3 to 119593
Data columns (total 7 columns):
paper_id            105889 non-null object
abstract            105889 non-null object
body_text           105889 non-null object
authors             105889 non-null object
title               105889 non-null object
journal             105889 non-null object
abstract_summary    105889 non-null object
dtypes: object(7)
memory usage: 6.5+ MB


In [17]:

df_covid = df_covid.head(12500)

In [18]:

import re

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

In [19]:
def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_covid['body_text'] = df_covid['body_text'].apply(lambda x: lower_case(x))
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: lower_case(x))

In [20]:
df_covid.head(4)


,paper_id,abstract,body_text,authors,title,journal,abstract_summary
3,372947b39005f7105cf7ecd4ce618174a655f9be,in this paper we study the effectiveness of th...,in december 2019 a novel strand of coronavirus...,"Cooper, Ian. Mondal, Argha...",A SIR model assumption for the spread of<br>C...,Chaos Solitons Fractals,"In this paper, we study the effectiveness of ..."
4,11d016e4272c8a89fe7986afde61e6fb91908304,interferences between pathogenic bacteria and ...,otitis media is a common childhood disease it ...,"Chonmaitree, Tasnee. Jennings, Kristofer...",Nasopharyngeal microbiota in infants and<br>c...,PLoS One,Interferences between pathogenic bacteria<br>...
5,72e7cb2184c9622cb852a98a88f3ed091d97bd00,,travel can provide access to employment and ed...,"Mindell, Jennifer S.. Ergler, Christina...",Taking the bus? Barriers and facilitators for...,Travel Behav Soc,Not provided.
6,d179f9a68fc31e1c2da3c0af5328f9fa093b979c,,to infectious diseases considerably more death...,"Mak, Tak W.. Saunders, Mary E.",Immunity to Pathogens,The Immune Response,Not provided.


In [24]:
os.makedirs("Data")
df_covid.to_csv("Data/covid.csv")

In [25]:
df_covid_test = pd.read_csv("Data/covid.csv")
text = df_covid_test.drop(["authors", "journal", "Unnamed: 0"], axis=1)
text.head(5)

,paper_id,abstract,body_text,title,abstract_summary
0,372947b39005f7105cf7ecd4ce618174a655f9be,in this paper we study the effectiveness of th...,in december 2019 a novel strand of coronavirus...,A SIR model assumption for the spread of<br>C...,"In this paper, we study the effectiveness of ..."
1,11d016e4272c8a89fe7986afde61e6fb91908304,interferences between pathogenic bacteria and ...,otitis media is a common childhood disease it ...,Nasopharyngeal microbiota in infants and<br>c...,Interferences between pathogenic bacteria<br>...
2,72e7cb2184c9622cb852a98a88f3ed091d97bd00,NaN,travel can provide access to employment and ed...,Taking the bus? Barriers and facilitators for...,Not provided.
3,d179f9a68fc31e1c2da3c0af5328f9fa093b979c,NaN,to infectious diseases considerably more death...,Immunity to Pathogens,Not provided.
4,57b8b330da231d213323e2aefd28efd74ca6cddd,NaN,direct detection of viral antigens and virussp...,Molecular Techniques for Blood and Blood<br>P...,Not provided.


In [26]:
text_dict = text.to_dict()
len_text = len(text_dict["paper_id"])


In [27]:
paper_id_list  = []
body_text_list = []

title_list = []
abstract_list = []
abstract_summary_list = []
for i in tqdm(range(0,len_text)):
  paper_id = text_dict["paper_id"][i]
  body_text = text_dict["body_text"][i].split("\n")
  title = text_dict["title"][i]
  abstract = text_dict["abstract"][i]
  abstract_summary = text_dict["abstract_summary"][i]
  for b in body_text:
    paper_id_list.append(paper_id)
    body_text_list.append(b)
    title_list.append(title)
    abstract_list.append(abstract)
    abstract_summary_list.append(abstract_summary)

100%|██████████| 12500/12500 [00:00<00:00, 13802.65it/s]


In [29]:
df_sentences = pd.DataFrame({"paper_id":paper_id_list},index=body_text_list)
df_sentences.to_csv("Data/covid_sentences.csv")
df_sentences.head()

,paper_id
in december 2019 a novel strand of coronavirus sarscov2 was identified in wuhan hubei province china causing a severe and potentially fatal respiratory syndrome ie covid19 since then it has become a pandemic declared by world health organization who on march 11 which has spread around the globe [1 2 3 4 5] who published in its website preliminary guidelines with public health care for the countries to deal with the 5 pandemic [6] since then the infectious disease has become a public health threat italy and usa are severely affected by covid19 [7 8 9] millions of people are forced by national governments to stay in selfisolation and in difficult conditions the disease is growing fast in many countries around the world in the absence of availability of a proper medicine or vaccine currently social distancing selfquarantine and wearing a face mask have been emerged as the most widelyused strategy for the mitigation and control of the pandemic 10 in this context mathematical models are required to estimate disease transmission recovery deaths and other significant parameters separately for various countries that is for different specific regions of high to low reported cases of covid19 different countries have already taken precise and differentiated measures that are important to control the spread of the disease however still now important factors such as population density insufficient evidence for different symptoms transmission mechanism and unavailability of a proper vaccine makes it difficult 15 to deal with such a highly infectious and deadly disease especially in high population density countries such as india [10 11 12] recently many research articles have adopted the modelling approach using real incidence datasets from affected countries and have investigated different characteristics as a function of various parameters of the outbreak and the effects of intervention strategies in different countries respective to their current situations,372947b39005f7105cf7ecd4ce618174a655f9be
it is imperative that mathematical models are developed to provide insights and make predictions about the 20 pandemic to plan effective control strategies and policies [13 14 15] modelling approaches [8 16 17 18 19 20 21] are helpful to understand and predict the possibility and severity of the disease outbreak and provide key information to determine the intensity of covid19 disease intervention the susceptibleinfectedremoved sir model and its extended modifications [22 23 24 25] such as the extendedsusceptibleinfectedremoved esir mathematical model in various forms have been used in previous studies [26 27 28] to model the spread of 25 covid19 within communities,372947b39005f7105cf7ecd4ce618174a655f9be
here we propose the use of a novel sir model with different characteristics one of the major assumptions of the classic sir model is that there is a homogeneous mixing of the infected and susceptible populations and that the total population is constant in time in the classic sir model the susceptible population decreases monotonically towards zero however these assumptions are not valid in the case of the spread of the covid19 virus since new 30 epicentres spring up around the globe at different times to account for this the sir model that we propose here does not consider the total population and takes the susceptible population as a variable that can be adjusted at various times to account for new infected individuals spreading throughout a community resulting in an increase in the susceptible population ie to the socalled surges the sir model we introduce here is given by the same simple system of three ordinary differential equations odes with the classic sir model and can be used to gain 35 a better understanding of how the virus spreads within a community of variable populations in time when surges occur importantly it can be used to make predictions of the number of infections and deaths that may occur in the future and provi

In [30]:
f_sentences = pd.DataFrame({"paper_id":paper_id_list,"title":title_list,"abstract":abstract_list,"abstract_summary":abstract_summary_list},index=body_text_list)
df_sentences.to_csv("Data/covid_sentences_Full.csv")
df_sentences.head()

,paper_id
in december 2019 a novel strand of coronavirus sarscov2 was identified in wuhan hubei province china causing a severe and potentially fatal respiratory syndrome ie covid19 since then it has become a pandemic declared by world health organization who on march 11 which has spread around the globe [1 2 3 4 5] who published in its website preliminary guidelines with public health care for the countries to deal with the 5 pandemic [6] since then the infectious disease has become a public health threat italy and usa are severely affected by covid19 [7 8 9] millions of people are forced by national governments to stay in selfisolation and in difficult conditions the disease is growing fast in many countries around the world in the absence of availability of a proper medicine or vaccine currently social distancing selfquarantine and wearing a face mask have been emerged as the most widelyused strategy for the mitigation and control of the pandemic 10 in this context mathematical models are required to estimate disease transmission recovery deaths and other significant parameters separately for various countries that is for different specific regions of high to low reported cases of covid19 different countries have already taken precise and differentiated measures that are important to control the spread of the disease however still now important factors such as population density insufficient evidence for different symptoms transmission mechanism and unavailability of a proper vaccine makes it difficult 15 to deal with such a highly infectious and deadly disease especially in high population density countries such as india [10 11 12] recently many research articles have adopted the modelling approach using real incidence datasets from affected countries and have investigated different characteristics as a function of various parameters of the outbreak and the effects of intervention strategies in different countries respective to their current situations,372947b39005f7105cf7ecd4ce618174a655f9be
it is imperative that mathematical models are developed to provide insights and make predictions about the 20 pandemic to plan effective control strategies and policies [13 14 15] modelling approaches [8 16 17 18 19 20 21] are helpful to understand and predict the possibility and severity of the disease outbreak and provide key information to determine the intensity of covid19 disease intervention the susceptibleinfectedremoved sir model and its extended modifications [22 23 24 25] such as the extendedsusceptibleinfectedremoved esir mathematical model in various forms have been used in previous studies [26 27 28] to model the spread of 25 covid19 within communities,372947b39005f7105cf7ecd4ce618174a655f9be
here we propose the use of a novel sir model with different characteristics one of the major assumptions of the classic sir model is that there is a homogeneous mixing of the infected and susceptible populations and that the total population is constant in time in the classic sir model the susceptible population decreases monotonically towards zero however these assumptions are not valid in the case of the spread of the covid19 virus since new 30 epicentres spring up around the globe at different times to account for this the sir model that we propose here does not consider the total population and takes the susceptible population as a variable that can be adjusted at various times to account for new infected individuals spreading throughout a community resulting in an increase in the susceptible population ie to the socalled surges the sir model we introduce here is given by the same simple system of three ordinary differential equations odes with the classic sir model and can be used to gain 35 a better understanding of how the virus spreads within a community of variable populations in time when surges occur importantly it can be used to make predictions of the number of infections and deaths that may occur in the future and provi

In [36]:
#Data Embedding


import pandas as pd
from tqdm import tqdm

df_sentences = pd.read_csv("Data/covid_sentences.csv")
df_sentences = df_sentences.set_index("Unnamed: 0")

In [37]:

df_sentences = df_sentences["paper_id"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)



361001

In [38]:
list(df_sentences.keys())[:5]

['in december 2019 a novel strand of coronavirus sarscov2 was identified in wuhan hubei province china causing a severe and potentially fatal respiratory syndrome ie covid19 since then it has become a pandemic declared by world health organization who on march 11 which has spread around the globe [1 2 3 4 5]  who published in its website preliminary guidelines with public health care for the countries to deal with the 5 pandemic [6]  since then the infectious disease has become a public health threat italy and usa are severely affected by covid19 [7 8 9]  millions of people are forced by national governments to stay in selfisolation and in difficult conditions the disease is growing fast in many countries around the world in the absence of availability of a proper medicine or vaccine currently social distancing selfquarantine and wearing a face mask have been emerged as the most widelyused strategy for the mitigation and control of the pandemic 10 in this context mathematical models ar

In [39]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 361001/361001 [00:00<00:00, 1129094.10it/s]


In [40]:
df = pd.read_csv("Data/covid_sentences_Full.csv", index_col=0)
df.head()

,paper_id
in december 2019 a novel strand of coronavirus sarscov2 was identified in wuhan hubei province china causing a severe and potentially fatal respiratory syndrome ie covid19 since then it has become a pandemic declared by world health organization who on march 11 which has spread around the globe [1 2 3 4 5] who published in its website preliminary guidelines with public health care for the countries to deal with the 5 pandemic [6] since then the infectious disease has become a public health threat italy and usa are severely affected by covid19 [7 8 9] millions of people are forced by national governments to stay in selfisolation and in difficult conditions the disease is growing fast in many countries around the world in the absence of availability of a proper medicine or vaccine currently social distancing selfquarantine and wearing a face mask have been emerged as the most widelyused strategy for the mitigation and control of the pandemic 10 in this context mathematical models are required to estimate disease transmission recovery deaths and other significant parameters separately for various countries that is for different specific regions of high to low reported cases of covid19 different countries have already taken precise and differentiated measures that are important to control the spread of the disease however still now important factors such as population density insufficient evidence for different symptoms transmission mechanism and unavailability of a proper vaccine makes it difficult 15 to deal with such a highly infectious and deadly disease especially in high population density countries such as india [10 11 12] recently many research articles have adopted the modelling approach using real incidence datasets from affected countries and have investigated different characteristics as a function of various parameters of the outbreak and the effects of intervention strategies in different countries respective to their current situations,372947b39005f7105cf7ecd4ce618174a655f9be
it is imperative that mathematical models are developed to provide insights and make predictions about the 20 pandemic to plan effective control strategies and policies [13 14 15] modelling approaches [8 16 17 18 19 20 21] are helpful to understand and predict the possibility and severity of the disease outbreak and provide key information to determine the intensity of covid19 disease intervention the susceptibleinfectedremoved sir model and its extended modifications [22 23 24 25] such as the extendedsusceptibleinfectedremoved esir mathematical model in various forms have been used in previous studies [26 27 28] to model the spread of 25 covid19 within communities,372947b39005f7105cf7ecd4ce618174a655f9be
here we propose the use of a novel sir model with different characteristics one of the major assumptions of the classic sir model is that there is a homogeneous mixing of the infected and susceptible populations and that the total population is constant in time in the classic sir model the susceptible population decreases monotonically towards zero however these assumptions are not valid in the case of the spread of the covid19 virus since new 30 epicentres spring up around the globe at different times to account for this the sir model that we propose here does not consider the total population and takes the susceptible population as a variable that can be adjusted at various times to account for new infected individuals spreading throughout a community resulting in an increase in the susceptible population ie to the socalled surges the sir model we introduce here is given by the same simple system of three ordinary differential equations odes with the classic sir model and can be used to gain 35 a better understanding of how the virus spreads within a community of variable populations in time when surges occur importantly it can be used to make predictions of the number of infections and deaths that may occur in the future and provi

In [3]:
#Courtesy: https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')


corpus = df_sentences_list
#corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)
#Uncomment above line to train BERT embedding on entire corpus: Estimated time >4 hours. 
"""Sample corpus embedding created using subset of data: corpus_embedding.pkl, link shared in README"""

NameError: name 'df_sentences_list' is not defined

In [ ]:
import pickle as pkl

emb=open('corpus_embedding','wb')
pkl.dump(corpus_embeddings,emb)
emb.close()
    
    